<a href="https://colab.research.google.com/github/SuperNZH/Deep-Learning-Practice/blob/main/Dive%20in%20DL/Linear_regression_byFrameworkPower__).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *Linear Regression Simple Implementation*

# About Dataset

In [1]:
%matplotlib inline
import torch
from torch import nn
from IPython import display
from matplotlib import pyplot as plt
import numpy as np
import random

In [2]:
num_inputs = 2
num_examples = 1000
true_w = [2, -3.4]
true_b = 4.2
features = torch.tensor(np.random.normal(0, 1, (num_examples, num_inputs)), dtype=torch.float)
labels = true_w[0] * features[:, 0] + true_w[1] * features[:, 1] + true_b
labels += torch.tensor(np.random.normal(0, 0.01, size=labels.size()), dtype=torch.float)

In [3]:
import torch.utils.data as Data

batch_size = 10
dataset = Data.TensorDataset(features, labels)
data_iter = Data.DataLoader(dataset, batch_size, shuffle=True)

# TensorDataset把数据集包装起来，便于调用

In [4]:
for X, y in data_iter:
    print(X,y)
    break

tensor([[-0.1887,  1.1698],
        [-0.5246,  0.0104],
        [-0.1778,  0.4116],
        [-0.8547, -0.7210],
        [ 1.6877, -1.0059],
        [ 1.2162, -0.2211],
        [ 0.6482,  0.5981],
        [-0.6620, -0.9500],
        [ 0.1989,  0.4419],
        [ 2.4849,  1.1490]]) tensor([-0.1537,  3.1277,  2.4385,  4.9227, 10.9896,  7.3997,  3.4600,  6.1331,
         3.0875,  5.2412])


In [5]:
# 如果pytorch来定义模型，只需关注使用哪些层来构造模型

class LinearNet(nn.Module):
  def __init__(self, n_feature):
    super(LinearNet, self).__init__()
    self.linear = nn.Linear(n_feature, 1)

  def forward(self, x):
    y = self.linear(x)
    return y


In [6]:
# 另外更简略的写法
# lin_net = nn.Sequential(
#     nn.Linear(num_inputs, 1)
# )

# print(lin_net)
# print(lin_net[0])

In [7]:
net = LinearNet(num_inputs)
print(net)
# torch.nn.Linear(in_features, out_features, bias=True, device=None, dtype=None)
# forward 定义前向传播

LinearNet(
  (linear): Linear(in_features=2, out_features=1, bias=True)
)


In [8]:
# 可以通过net.parameters()来查看模型所有的可学习参数，此函数将返回一个生成器
for param in net.parameters():
  print(param)

Parameter containing:
tensor([[-0.1659,  0.3763]], requires_grad=True)
Parameter containing:
tensor([-0.5918], requires_grad=True)


In [9]:
# 初始化模型参数
# 在使用net前，需要初始化模型参数，如线性回归模型中的权重和偏差
# 通过init.normal_将权重参数每个元素初始化为随机采样于均值为0、标准差为0.01的正态分布。
# 偏差会初始化为零。

from torch.nn import init

init.normal_(net.linear.weight, mean=0, std=0.01)
init.constant_(net.linear.bias, val=0)

'''
# for sequential的
init.normal_(net[0].weight, mean=0, std=0.01)
init.constant_(net[0].bias, val=0)
'''

'\n# for sequential的\ninit.normal_(net[0].weight, mean=0, std=0.01)\ninit.constant_(net[0].bias, val=0)\n'

In [10]:
# 定义损失函数

loss = nn.MSELoss()

In [11]:
# 定义optimization的算法

import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr=0.03)
print(optimizer)

# 把学习到的parameters丢进去不断验证并优化
# 此外这个SGD方法还可以针对不同的param指定不同的learning rate
'''
optimizer =optim.SGD([
                {'params': net.subnet1.parameters()}, # lr=0.03
                {'params': net.subnet2.parameters(), 'lr': 0.01}
            ], lr=0.03)
'''

SGD (
Parameter Group 0
    dampening: 0
    lr: 0.03
    momentum: 0
    nesterov: False
    weight_decay: 0
)


"\noptimizer =optim.SGD([\n                {'params': net.subnet1.parameters()}, # lr=0.03\n                {'params': net.subnet2.parameters(), 'lr': 0.01}\n            ], lr=0.03)\n"

In [12]:
# 调整学习率

# for param_group in optimizer.param_groups:
#   param_group['lr'] *= 0.1

In [13]:
# 训练模型

num_epochs = 3
for epoch in range(1, num_epochs+1):
  for X,y in data_iter:
    output = net(X)
    # -1是指动态调整
    l = loss(output,y.view(-1,1))
    # 每次都要清空梯度
    optimizer.zero_grad()
    l.backward()
    optimizer.step()
  print('epoch %d, loss: %f' %(epoch, l.item()))

epoch 1, loss: 0.000240
epoch 2, loss: 0.000166
epoch 3, loss: 0.000130


In [18]:
dense = net.linear
print(true_w, dense.weight)
print('--'*20)
print(true_b, dense.bias)

[2, -3.4] Parameter containing:
tensor([[ 1.9987, -3.4000]], requires_grad=True)
----------------------------------------
4.2 Parameter containing:
tensor([4.2009], requires_grad=True)
